In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
from tensorflow import keras
import cv2
from PIL import Image
from tensorflow.keras import models, layers
from glob import glob # ファイルパスを一気に取得するためのライブラリ
from sklearn.model_selection import train_test_split

In [24]:
cat_filepath = glob('data/cat/*jpg') # * で全ての意味

In [25]:
dog_filepath = glob('data/dog/*jpg')

In [26]:
import os
import random

def reset_seed(seed=0):

    os.environ['PYTHONHASHSEED'] = '0'
    random.seed(seed) #　random関数のシードを固定
    np.random.seed(seed) #numpyのシードを固定
    tf.random.set_seed(seed) #tensorflowのシードを固定

In [27]:
x, t = [], []

for filepath in dog_filepath:
    img = Image.open(filepath)
    x.append(np.array(img))
    t.append(0)
    
for filepath in cat_filepath:
    img = Image.open(filepath)
    x.append(np.array(img))
    t.append(1)

In [28]:
len(x), len(t)

(500, 500)

In [29]:
x = np.array(x)/255
t = np.array(t)

In [30]:
x.shape

(500, 224, 224, 3)

In [31]:
t.shape

(500,)

In [32]:
x_train, x_val, t_train, t_val = train_test_split(x, t, test_size=0.3, random_state=0)
len(x_train), len(x_val), len(t_train), len(t_val)

(350, 150, 350, 150)

In [33]:
x_train.shape

(350, 224, 224, 3)

In [34]:
x[0].shape

(224, 224, 3)

In [13]:
# from tensorflow.keras.applications import ResNet50

In [14]:
# resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [48]:
# resnet.summary()

In [16]:
# from tensorflow.keras import models, layers
# reset_seed(0)

# model = models.Sequential()

# for layer in resnet.layers:
#     layer.trainable = False
    
# model.add(resnet)
# model.add(layers.Flatten())
# model.add(layers.BatchNormalization())
# model.add(layers.Dense(100, activation='relu'))
# model.add(layers.Dense(30, activation='relu'))
# model.add(layers.Dense(2, activation='softmax'))

In [17]:
# optimizer = keras.optimizers.Adam(0.005)

# model.compile(loss='sparse_categorical_crossentropy',
#              optimizer=optimizer, metrics=['accuracy'])

In [45]:
# model.summary()

In [46]:
# history = model.fit(x_train, t_train,
#                    batch_size=128,
#                    epochs=30,
#                    validation_data=(x_val, t_val))

In [47]:
# results = pd.DataFrame(history.history)
# results[['accuracy', 'val_accuracy']].plot()

In [21]:
# Filter 11x11 --> Filter 5x5 --> Filter 3x3 *3 layers
# Stride 4
# BatchNorm

In [50]:
reset_seed(0)

model = models.Sequential()

model.add(layers.Conv2D(64, (3, 3, 3), activation='relu', strides=(1, 1), 
                        padding=(1, 1, 1, 1), input_shape=(224, 224, 3)))
model.add(layers.Conv2D(64, (3, 3, 64), activation='relu', strides=(1, 1),
                        padding=(1, 1, 1, 1)))
model.add(layers.MaxPool2D((2, 2), padding=(0, 0, 0, 0)))
model.add(layers.Conv2D(64, (3, 3, 64), activation='relu', strides=(1, 1), 
                        padding=(1, 1, 1, 1)))
model.add(layers.Conv2D(64, (3, 3, 128), activation='relu', strides=(1, 1), 
                        padding=(1, 1, 1, 1)))
model.add(layers.MaxPool2D((2, 2), padding=(0, 0, 0, 0)))
model.add(layers.Conv2D(64, (5, 5), activation='relu', strides=(2, 2)))
model.add(layers.MaxPool2D((2, 2)))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPool2D((2, 2)))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPool2D((2, 2)))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPool2D((2, 2)))
model.add(layers.BatchNormalization())

model.add(layers.Flatten())
model.add(layers.Dense(100, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dense(33, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dense(2, activation='softmax'))

opt = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)

model.compile(loss='sparse_categorical_crossentropy', optimizer=opt,
             metrics=['accuracy'])

ValueError: The `kernel_size` argument must be a tuple of 2 integers. Received: (3, 3, 3)

In [44]:
history = model.fit(x_train, t_train,
                   batch_size=128,
                   epochs=30,
                   validation_data=(x_val, t_val))

Train on 350 samples, validate on 150 samples
Epoch 1/30
350/350 [==============================] - 5s 13ms/sample - loss: 0.9014 - accuracy: 0.5486 - val_loss: 0.6916 - val_accuracy: 0.5333
Epoch 2/30
350/350 [==============================] - 2s 7ms/sample - loss: 0.7714 - accuracy: 0.5914 - val_loss: 0.7147 - val_accuracy: 0.4933
Epoch 3/30
350/350 [==============================] - 3s 8ms/sample - loss: 0.5382 - accuracy: 0.7143 - val_loss: 0.8495 - val_accuracy: 0.5067
Epoch 4/30
350/350 [==============================] - 3s 8ms/sample - loss: 0.3693 - accuracy: 0.8457 - val_loss: 1.0041 - val_accuracy: 0.5067
Epoch 5/30
350/350 [==============================] - 3s 7ms/sample - loss: 0.2992 - accuracy: 0.9143 - val_loss: 1.0858 - val_accuracy: 0.5067
Epoch 6/30
350/350 [==============================] - 3s 8ms/sample - loss: 0.2287 - accuracy: 0.9600 - val_loss: 1.1195 - val_accuracy: 0.5067
Epoch 7/30
350/350 [==============================] - 3s 7ms/sample - loss: 0.1754 - accu